In [5]:
from pyfirmata import Arduino
import serial
import time

In [18]:
class MCSArduino(Arduino):
    pin_diode = 13
    pin_TTL = 6
    pin_LED_lights = 7
    pin_feeder = 8
    
    def __init__(self, *args, **kwargs):
        self.last_cmd = False  # False - Arduino LOW, True - Arduino HIGH
        self.is_light_off = False
        super(MCSArduino, self).__init__(*args, **kwargs)
        
    def start_or_stop(self):  # acquisition
        self.last_cmd = not self.last_cmd
        self.digital[MCSArduino.pin_diode].write(self.last_cmd)
        self.digital[MCSArduino.pin_TTL].write(self.last_cmd)

    def switch_light(self):
        self.is_light_off = not self.is_light_off
        self.digital[MCSArduino.pin_LED_lights].write(self.is_light_off)
        
    def feed(self):
        self.digital[MCSArduino.pin_feeder].write(True)
        time.sleep(0.01)
        self.digital[MCSArduino.pin_feeder].write(False)
        
        
class FakeArduino():
    def __init__(self):
        self.is_light_off = False
        
    def start_or_stop(self):
        print("Fake Arduino - sending a TTL pulse")
        
    def exit(self):
        print("Fake Arduino - exiting...")
        
    def switch_light(self):
        self.is_light_off = not self.is_light_off
        print("Fake Arduino - switching light on/off...")
        
    def feed(self):
        print("Fake Arduino - feeding...")

In [7]:
class FakeDevice:
    def write(self):
        pass
    def close(self):
        pass

class Feeder:
    def __init__(self, port, baudrate=9600):
        if port == 'fake':
            self.device = FakeDevice()
        else:
            self.device = serial.Serial(port, baudrate=baudrate)
        
    def feed(self):
        self.device.write('SP200\r'.encode())
        self.device.write('LR6175\r'.encode())
        self.device.write('m\r'.encode())
        
    def exit(self):
        self.device.close()

### Testing Arduino TTL pulses

In [15]:
board = MCSArduino('COM10')  # Windows - 'COM10', Linux - '/dev/ttyACM0', check /dev/tty*

In [10]:
# check the light on-off
board.switch_light()

In [20]:
# testing sync with acquisition system
board.start_or_stop()

In [16]:
for i in range(10):
    time.sleep(5)
    board.feed()

In [5]:
# Arduino should blink 2 times
for i in range(4):
    board.start_or_stop()
    time.sleep(2)

In [14]:
board.digital[5].value

False

In [17]:
board.exit()

### Testing feeder

In [4]:
feeder = Feeder('COM8')

time.sleep(5)  # time to get inside the chamber and see that it works
feeder.feed()

feeder.exit()